# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,11.40,95,100,2.88,AU,1724542140
1,1,port-aux-francais,-49.3500,70.2167,2.60,94,97,14.81,TF,1724542142
2,2,yellowknife,62.4560,-114.3525,17.45,59,75,3.09,CA,1724542143
3,3,loango,-4.6719,11.8312,21.48,88,15,1.25,CG,1724542144
4,4,stanley,54.8680,-1.6985,11.01,80,20,6.17,GB,1724542114


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)    

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df
#Max lower than 21, higher than 18.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 21) & (ideal_weather_df["Max Temp"] > 19)]
#Wind speed less than 10mph
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]
#Cloudiness lower than 30
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] < 30]
#Humidity lower than 60
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Humidity"] < 60]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,port hedland,-20.3167,118.5667,19.03,59,0,3.09,AU,1724542225
124,124,karratha,-20.7377,116.8463,20.13,45,0,4.62,AU,1724541801
168,168,birjand,32.8663,59.2211,20.48,27,0,7.56,IR,1724541851
259,259,miandrivazo,-19.5167,45.4667,20.71,53,5,1.49,MG,1724542442
343,343,svetlyy,54.6750,20.1347,20.47,56,0,4.31,RU,1724542536
363,363,morondava,-20.2833,44.2833,20.18,59,1,4.71,MG,1724542558
374,374,tukzar,35.9483,66.4213,19.37,23,0,2.91,AF,1724542573
463,463,kamyshin,50.0983,45.4160,20.78,40,26,1.82,RU,1724542679
465,465,wysokie mazowieckie,52.9166,22.5171,19.67,54,8,3.81,PL,1724542681
555,555,lucapa,-8.4192,20.7447,19.59,29,3,1.85,AO,1724542791


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_weather_df, columns = ["City", "Country", "Lat", "Lng", "Humidity"])
                                                                  
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
72,port hedland,AU,-20.3167,118.5667,59,
124,karratha,AU,-20.7377,116.8463,45,
168,birjand,IR,32.8663,59.2211,27,
259,miandrivazo,MG,-19.5167,45.4667,53,
343,svetlyy,RU,54.6750,20.1347,56,
363,morondava,MG,-20.2833,44.2833,59,
374,tukzar,AF,35.9483,66.4213,23,
463,kamyshin,RU,50.0983,45.4160,40,
465,wysokie mazowieckie,PL,52.9166,22.5171,54,
555,lucapa,AO,-8.4192,20.7447,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port hedland - nearest hotel: The Esplanade Hotel
karratha - nearest hotel: Karratha International Hotel
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
miandrivazo - nearest hotel: Baobab
svetlyy - nearest hotel: Гостиница Спортивная
morondava - nearest hotel: Hotel Sharon Menabe
tukzar - nearest hotel: No hotel found
kamyshin - nearest hotel: Глория
wysokie mazowieckie - nearest hotel: Hotel Gutland
lucapa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
72,port hedland,AU,-20.3167,118.5667,59,The Esplanade Hotel
124,karratha,AU,-20.7377,116.8463,45,Karratha International Hotel
168,birjand,IR,32.8663,59.2211,27,مهمانسرای جهانگردی بیرجند
259,miandrivazo,MG,-19.5167,45.4667,53,Baobab
343,svetlyy,RU,54.6750,20.1347,56,Гостиница Спортивная
363,morondava,MG,-20.2833,44.2833,59,Hotel Sharon Menabe
374,tukzar,AF,35.9483,66.4213,23,No hotel found
463,kamyshin,RU,50.0983,45.4160,40,Глория
465,wysokie mazowieckie,PL,52.9166,22.5171,54,Hotel Gutland
555,lucapa,AO,-8.4192,20.7447,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
        "Lng",
        "Lat",
        geo = True,
        tiles = "OSM",
        frame_width = 700,
        frame_height = 500,
        scale = 0.01,
        color = "City",
        hover_cols=["Humidity", "Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)